In [1]:
import langchain
print(langchain.__version__)

0.3.4


In [ ]:
from langchain_openai import ChatOpenAI
import os
from typing import Union
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=300,
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Pydantic 모델 정의
class CommonConversation(BaseModel):
     """Text to SQL Prompt Instruction"""
     common_conversation: str = Field(
        description = """
        너는 "누구나데이터"의 직원으로써, 사용자를 친절하게 응대해줘.
        사용자가 어려움을 겪으면 사업자번호인 010-1234-5678로 응대해줘.
        """
    )

class TextToSQL(BaseModel):
    """Common conversation Prompt Instruction"""
    TextToSQL: str = """
        Please write ONLY the SQL query according to this schema.
    """
    difficulty: str = Field(
        default="easy",
        enum=["easy", "normal", "hard"],
        description="""
        Please measure the SQL conversion difficulty for this request. The difficulty levels are easy normal hard.
        """
    )

class Response(BaseModel):
    output: Union[CommonConversation, TextToSQL]

# 구조화된 출력 LLM 구성
structured_llm = llm.with_structured_output(Response)



In [29]:
# 의사 분류기 테스트
result = structured_llm.invoke("저번주에 로그인한 유저수")
result

Response(output=CommonConversation(TextToSQL='SELECT COUNT(*) FROM users WHERE last_login >= NOW() - INTERVAL 7 DAY;', difficulty='easy'))

In [30]:
result = structured_llm.invoke("Tell me a joke about cats")
result

Response(output=TextToSQL(common_conversation='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!'))

In [31]:
result = structured_llm.invoke("저번주에 로그인한 유저가 마음에 안들어.")
result

Response(output=TextToSQL(common_conversation='안녕하세요! 어떤 문제가 있으신가요? 저희가 도와드릴 수 있는 부분이 있다면 말씀해 주세요. 만약 특정 유저에 대한 문제가 심각하다면, 사업자번호 010-1234-5678로 연락해 주시면 더욱 자세히 도와드리겠습니다.'))